In [7]:
import os
import elasticsearch as es
from dotenv import load_dotenv

In [8]:
load_dotenv()

True

In [9]:
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
client = es.Elasticsearch("http://10.11.12.69:9200", http_auth=(username, password))

In [10]:
client.ping()

True

In [11]:
client.indices.get_alias()

/usr/local/Caskroom/miniconda/base/envs/ADP/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: this request accesses system indices: [.security-7, .kibana_7.15.1_001, .async-search, .kibana_security_session_1, .apm-agent-configuration, .apm-custom-link, .fleet-policies-7, .tasks, .kibana_task_manager_7.15.1_001], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)
/usr/local/Caskroom/miniconda/base/envs/ADP/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: this request accesses aliases with names reserved for system indices: [.kibana_task_manager_7.15.1, .kibana_7.15.1, .kibana_task_manager, .security, .kibana, .fleet-policies], but in a future major version, direct access to system indices and their aliases will not be allowed
  warnings.warn(message, category=ElasticsearchWarning)


{'.security-7': {'aliases': {'.security': {}}},
 'kibana_sample_data_logs': {'aliases': {}},
 '.monitoring-kibana-7-2021.11.21': {'aliases': {}},
 '.kibana_7.15.1_001': {'aliases': {'.kibana': {}, '.kibana_7.15.1': {}}},
 'logstash-2021.11.08-000001': {'aliases': {'logstash': {'is_write_index': True}}},
 'kibana_sample_data_flights': {'aliases': {}},
 '.async-search': {'aliases': {}},
 '.monitoring-logstash-7-2021.11.20': {'aliases': {}},
 '.kibana_security_session_1': {'aliases': {}},
 '.apm-agent-configuration': {'aliases': {}},
 '.monitoring-es-7-2021.11.20': {'aliases': {}},
 '.apm-custom-link': {'aliases': {}},
 '.monitoring-es-7-2021.11.17': {'aliases': {}},
 '.monitoring-es-7-2021.11.24': {'aliases': {}},
 '.monitoring-logstash-7-2021.11.23': {'aliases': {}},
 'students_kismet_devices': {'aliases': {}},
 '.monitoring-kibana-7-2021.11.23': {'aliases': {}},
 '.monitoring-kibana-7-2021.11.17': {'aliases': {}},
 '.monitoring-kibana-7-2021.11.18': {'aliases': {}},
 '.fleet-policies-7

In [12]:
index_name = "constantin_amazondataproject_v2"

In [13]:
client.indices.create(index=index_name)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'constantin_amazondataproject_v2'}

In [14]:
client.search(index=index_name, query={"match_all": {}})

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [15]:
import pandas as pd

In [16]:
import csv
import numpy as np

In [17]:
convert_to_bool = lambda x: True if x == "Y" else False
converters = {"vine": convert_to_bool, "verified_purchase": convert_to_bool}

In [18]:
dtypes = {
    "marketplace": "category",
    "customer_id": np.int64,
    "review_id": "object",
    "product_id": "object",
    "product_parent": np.int64,
    "product_title": "object",
    "star_rating": np.int8,
    "helpful_votes": np.int16,
    "total_votes": np.int16,
    "review_headline": "object",
    "review_body": "object",
}

In [19]:
def data_iterator(reader):
        # need to remove when using csv.DictReader
        for chunk in reader:
            for _, row in chunk.iterrows():
                doc = {
                    "marketplace": row["marketplace"],
                    "customer_id": row["customer_id"],
                    "review_id": row["review_id"],
                    "product_id": row["product_id"],
                    "product_parent": row["product_parent"],
                    "product_title": row["product_title"],
                    "product_category": row["product_category"],
                    "star_rating": row["star_rating"],
                    "helpful_votes": row["helpful_votes"],
                    "total_votes": row["total_votes"],
                    "vine": row["vine"],
                    "verified_purchase": row["verified_purchase"],
                    "review_headline": row["review_headline"],
                    "review_body": row["review_body"],
                    "review_date": row["review_date"],
                }
                yield doc

In [91]:
client.delete_by_query(index=index_name, body={"query": {"match_all": {}}})

{'took': 7,
 'timed_out': False,
 'total': 98,
 'deleted': 98,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [92]:
client.search({"query": {"match_all": {}}}, index=index_name)

/var/folders/ps/dgdwqdr11mq34k0cc13_s_940000gn/T/ipykernel_98250/4064195043.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.search({"query": {"match_all": {}}}, index="constantin_amazondataproject")


{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [20]:
from elasticsearch.helpers import bulk
with open("../amazon_reviews_multilingual_DE_v1_00.tsv", mode="r") as f:
    # TODO: replace this with a version of csv.DictReader
    reader = pd.read_csv(
        f,
        delimiter="\t",
        quoting=csv.QUOTE_NONE,
        encoding="utf-8",
        chunksize=7000,
        parse_dates=["review_date"],
        converters=converters,
        dtype=dtypes,
    )
    res = bulk(client=client, index=index_name, actions=data_iterator(reader))
res

BulkIndexError: ('1 document(s) failed to index.', [{'index': {'_index': 'constantin_amazondataproject_v2', '_type': '_doc', '_id': 'NWNOUn0BVCy-8HZa6bPe', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse', 'caused_by': {'type': 'json_parse_exception', 'reason': "Non-standard token 'NaN': enable JsonParser.Feature.ALLOW_NON_NUMERIC_NUMBERS to allow\n at [Source: (ByteArrayInputStream); line: 1, column: 303]"}}, 'data': {'marketplace': 'DE', 'customer_id': 29422633, 'review_id': 'R2MD4CKWIMSZFG', 'product_id': 'B000UTMPHC', 'product_parent': 285766066, 'product_title': 'Saw III (Unrated)', 'product_category': 'Video DVD', 'star_rating': 5, 'helpful_votes': 1, 'total_votes': 3, 'vine': False, 'verified_purchase': False, 'review_headline': nan, 'review_body': 'Hallo,  also ich freu mich schon auf die DVD von SAW 3 :)  Ist echt ein super Film mit viel gemetzel ...  und ja es gibt einen teil 4, gibt auch schon einen Trailer dazu.  Der Film soll zu Hall